In [1]:
import transformers

In [2]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification

In [16]:
model_name='bert-base-cased'

In [1]:
import pandas as pd
import numpy as np

In [8]:
df=pd.read_csv(r"C:\ml\imdb\train.tsv\train.tsv",sep="\t")

In [10]:
maxlen=512

In [11]:
num_samples=len(df)

In [13]:
x_ids=np.zeros((num_samples,maxlen))

In [14]:
x_mask=np.zeros((num_samples,maxlen))

In [17]:
tokenizer=AutoTokenizer.from_pretrained(model_name)

C:\Users\krish\miniconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\krish\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [24]:
for i,phrase in enumerate(df['Phrase']):
    token=tokenizer(phrase,max_length=512,padding='max_length',truncation=True,add_special_tokens=True,return_tensors='tf')
    x_ids[i,:]=token['input_ids']
    x_mask[i,:]=token['attention_mask']

In [25]:
x_ids

array([[  101.,   138.,  1326., ...,     0.,     0.,     0.],
       [  101.,   138.,  1326., ...,     0.,     0.,     0.],
       [  101.,   138.,  1326., ...,     0.,     0.,     0.],
       ...,
       [  101.,   170., 25247., ...,     0.,     0.,     0.],
       [  101.,   170., 25247., ...,     0.,     0.,     0.],
       [  101., 22572., 12148., ...,     0.,     0.,     0.]])

In [26]:
x_mask

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]])

In [40]:
tokenizer.convert_ids_to_tokens([22573])

['therapist']

In [47]:
labels=np.zeros((num_samples,len(np.unique(df['Sentiment']))))
for i,j in enumerate(df['Sentiment']):
    labels[i][j]=1

In [45]:
np.unique(df['Sentiment'])

array([0, 1, 2, 3, 4], dtype=int64)

In [44]:
df.columns

Index(['PhraseId', 'SentenceId', 'Phrase', 'Sentiment'], dtype='object')

In [48]:
labels

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [63]:
import tensorflow as tf
data=tf.data.Dataset.from_tensor_slices((x_ids,x_mask,labels))

In [64]:
data.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.float64, name=None), TensorSpec(shape=(512,), dtype=tf.float64, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [65]:
def map_func(input_ids,masks,labels):
    return {'input_ids':input_ids,'attention_masks':masks},labels
    

In [66]:
data=data.map(map_func)

In [67]:
data.take(2)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.float64, name=None), 'attention_masks': TensorSpec(shape=(512,), dtype=tf.float64, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [68]:
batch_size=16
data=data.shuffle(10000).batch(batch_size,drop_remainder=True)

In [72]:
x=data.take(3)

In [81]:
split=0.9
size=int((num_samples/batch_size)*split)

In [82]:
size

8778

In [83]:
train_ds=data.take(size)
valid_ds=data.skip(size)

In [85]:
del data

In [86]:
from transformers import TFAutoModel

In [87]:
bert=TFAutoModel.from_pretrained(model_name)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [88]:
bert.summary()

Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  108310272 
                                                                 
Total params: 108,310,272
Trainable params: 108,310,272
Non-trainable params: 0
_________________________________________________________________


In [121]:
input_ids=tf.keras.layers.Input(shape=(512,),name='input_ids',dtype='int32')
masks=tf.keras.layers.Input(shape=(512,),name='attention_masks',dtype='int32')
embedding=bert.bert(input_ids,attention_mask=masks)[1]
x=tf.keras.layers.Dense(64,activation='relu')(embedding)
y=tf.keras.layers.Dense(labels.shape[1],activation='softmax')(x)


In [122]:
model=tf.keras.Model(inputs=(input_ids,masks),outputs=y)

In [123]:
model.layers[2].trainable=False

In [124]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_masks (InputLayer)   [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_masks[0][0]']        
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                         

In [125]:
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-5,decay=1e-6),loss='categorical_crossentropy',metrics=[tf.keras.metrics.CategoricalAccuracy('accuracy')])

In [126]:
hist=model.fit(train_ds,validation_data=valid_ds,epochs=3)

Epoch 1/3
   8/8778 [..............................] - ETA: 65:34:13 - loss: 1.7019 - accuracy: 0.1875

KeyboardInterrupt: 

In [135]:
def prepo(text):
    xt=tokenizer(text,max_length=512,padding='max_length',truncation=True,add_special_tokens=True,return_tensors='tf')
    return {'input_ids':xt['input_ids'],'attention_masks':xt['attention_mask']}

In [136]:
pred=prepo('hey this movie is awful')

In [137]:
model.predict(pred)

1/1 [==============================] - 1s 651ms/step


array([[0.03170261, 0.19619566, 0.21190512, 0.5039803 , 0.05621633]],
      dtype=float32)

<KerasTensor: shape=(None, 768) dtype=float32 (created by layer 'bert')>